### Multi-agent systems
- parallel
- sequential
- loop 


Why Multi-Agent Systems?
Types and why

1. **Modular Specialization (Role-Playing)** - By narrowing the scope for each agent, you increase the accuracy of that specific sub-task.

Even when using the same model (e.g., GPT-4o) for every agent, assigning distinct personas and system prompts reduces "distraction." A single prompt trying to be a coder, a security auditor, and a technical writer often suffers from "lost in the middle" context issues.


2. **Iterative Refinement (The Critic Loop)** - "self-correction" produces significantly higher quality than a single-pass prompt. A single agent is often "blind" to its own logical fallacies; a second agent acts as a fresh set of eyes.

Multi-agent systems allow for Adversarial or Collaborative loops. One agent generates an output, and another agent is specifically tasked with finding flaws in it.

3. **Parallelism and Scalability**
In a complex workflow, a multi-agent system can spin up several "worker" agents to perform tasks simultaneously.


4. **Diverse "State" Management** - "clean" context window. Instead of one massive prompt containing all tools and all data, each agent only carries the information relevant to its specific job.

Different agents can maintain different "memory" or "tools."

Agent A might have access to a Python interpreter.

Agent B might have access to a vector database (RAG).

Agent C might have access to a live API.

5. Error Isolation
If a single-agent prompt fails or hallucinates halfway through a 10-step process, the whole run is usually ruined. In a multi-agent system, you can implement checkpoints.

The Benefit: If the "Researcher Agent" fails to find data, the system can catch that error and retry that specific step before the "Writer Agent" ever starts.

In [103]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

In [104]:
# run in vs code only 

from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise RuntimeError("missing key")

In [ ]:
# do not run in VS code
# Setup for Colab

# set GOOGLE_API_KEY in Colab secrets
# the below retrieves the set key from the Colab
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

# helper function to be used 

# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")


In [105]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [ ]:

# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config
    ),
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")


✅ research_agent created.
✅ summarizer_agent created.


In [ ]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ research_agent created.
✅ summarizer_agent created.


In [ ]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


In [ ]:

runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the recent ways to evaluate agentic AI systems in finance? "
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False


DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.

You are an agent. Your internal name is "ResearchCoordinator".
-----------------------------------------------------------
Config:
{'http_options': {'headers': {'x-goog-api-client': 'google-adk/1.21.0 gl-python/3.12.11', 'user-agent': 'google-adk/1.21.0 gl-python/3.12.11'}}, 'tools': [{}]}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"What are the recent ways to evaluate agentic AI systems in finance? "}]


 ### Created new session: debug_session_id

User > What are the recent ways to evaluate agentic AI systems in finance? 


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 400, b'Bad Request', [(b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Type', b'application/json; charset=UTF-8'), (b'Content-Encoding', b'gzip'), (b'Date', b'Mon, 12 Jan 2026 09:14:57 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=517'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'), (b'Transfer-Encoding', b'chunked')])
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 400 Bad Request"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}

DEBUG:urllib3.connectionpool:https://telemetry.crewai.com:4319 "POST /v1/traces HTTP/1.1" 200 2


In [9]:
import logging

# Set logging to DEBUG to see the internal "handshakes" between agents
logging.basicConfig(level=logging.DEBUG)

In [ ]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the recent ways to evaluate agentic AI systems in finance? "
)

## Sequential Workflows

**Best practices **

- **Use placeholders**: `{topic}`, `{journal_guidelines}` in the instruction.
Pass concrete values in the inputs to the runner / Crew kickoff.
- Include a short, explicit checklist of journal constraints in `{journal_guidelines}` (word limits, section order, citation style, figure/table rules).
- Keep `temperature` low for deterministic outlines (e.g., 0–0.3).
- Validate the returned outline against the checklist and loop if missing constraints.
- Use `Process.sequential` / rate-limit handling as you already do with retry_config.

*Use a template (placeholders) in the agent instruction and pass the concrete values (topic, journal guidelines) via the runner / kickoff inputs. 
This keeps the agent prompt generic and lets you reuse it across topics and journals.*


**Centralized model + factory for multi-agent systems**

When defining multiple agents that use the same LLM (e.g., Gemini), use a central constant or small factory so you don't repeat literals — and prefer a factory (make_gemini) so you can easily vary `model`, `temperature`, or `retry options` per agent. 

*Avoid sharing the same live LLM instance if the SDK docs warn of internal state — use the factory to create instances.*


- Single SOURCE OF TRUTH for model name & retry config.
- Factory = easy per-agent overrides (critique vs drafting).
- Centralize deterministic settings (temperature, max_tokens) in one place.
- Avoid repeating code and typos.


In [ ]:
topic=
journal_guidelines= 

In [71]:
# Centralized model config + factory
MODEL_NAME = "gemini-2.5-flash-lite"

def make_gemini(model_name: str = MODEL_NAME, retry_options = retry_config):
    # return a fresh Gemini instance so each agent can have its own instance if needed
    return Gemini(model=model_name, retry_options=retry_options)

# default instance (or call make_gemini() per-agent)
GEMINI_DEFAULT = make_gemini()
# use GEMINI_DEFAULT instead of repeating Gemini(...)

#-----------------------------

planning_agent = Agent(
    name="PlanningAgent",
    model=GEMINI_DEFAULT,
    instruction="""Create an outline for an academic paper on a given topic as an integrative literature review.
Requirements:
- Provide a catchy title
- A short abstract (50-100 words)
- 5 keywords
- 3–5 main sections (e.g., Introduction, Methodology, Results/Discussion), each with bullet points of content
- A concluding thought suggesting an initial search query
- Apply formatting rules as specified in the provided editorial guidelines 
""",
    output_key="paper_outline",
)
print("✅ PlanningAgent created.")

#-----------------------------

research_agent = Agent(
    name="ResearchAgent",
    model=GEMINI_DEFAULT,
    instruction="""Use google_search tool to find relevant papers on Google Scholar
    Requirements:
    - Papers should be published within the last 2 years
    - Be in English""",
    tools=[google_search],
    output_key="research_findings",
)
print("✅ ResearchAgent created.")

#-----------------------------

# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model=GEMINI_DEFAULT,
    # The `{paper_outline}` and {research_findings} placeholders automatically inject the state value from the previous agent's output.
    instruction="""Following this outline strictly: {paper_outline}
    Use {research_findings}
    Write a brief, 200-word  research proposal""",
    output_key="paper_draft",  # The result of this agent will be stored with this key.
)
print("✅ WriterAgent created.")

#-----------------------------

# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model=GEMINI_DEFAULT,
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {paper_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  # This is the final output of the entire pipeline.
)

print("✅ EditorAgent created.")
#------------------

root_agent = SequentialAgent(
    name="PaperPipeline",
    sub_agents=[planning_agent, research_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ PlanningAgent created.
✅ ResearchAgent created.
✅ WriterAgent created.
✅ EditorAgent created.
✅ Sequential Agent created.


In [72]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a research brief about AI uses in Finance, abstract should be less than 150 words")

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Create an outline for an academic paper on a given topic as an integrative literature review.
Requirements:
- Provide a catchy title
- A short abstract (50-100 words)
- 5 keywords
- 3–5 main sections (e.g., Introduction, Methodology, Results/Discussion), each with bullet points of content
- A concluding thought suggesting an initial search query
- Apply formatting rules as specified in the provided editorial guidelines 


You are an agent. Your internal name is "PlanningAgent".
-----------------------------------------------------------
Config:
{'http_options': {'headers': {'x-goog-api-client': 'google-adk/1.21.0 gl-python/3.12.11', 'user-agent': 'google-adk/1.21.0 gl-python/3.12.11'}}}
----------------


 ### Created new session: debug_session_id

User > Write a research brief about AI uses in Finance, abstract should be less than 150 words


DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x17fbada60>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x1370499d0> server_hostname='generativelanguage.googleapis.com' timeout=None
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x308d4e540>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 400, b'Bad Request', [(b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Type', b'application/json; charset=UTF-8'), (b'Content-Encoding', b'gzip'), 

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}

DEBUG:urllib3.connectionpool:https://telemetry.crewai.com:4319 "POST /v1/traces HTTP/1.1" 200 2


Rate Limits:

Use Process.sequential instead of Process.hierarchical. This ensures only one agent calls the API at a time, keeping you under the 10-15 RPM (Requests Per Minute) limit.

Model Choice:

Use Gemini 2.0 Flash. It is significantly faster and has a higher daily request quota ($1,000$ RPD) than the "Pro" models ($100$ RPD).

Memory:

If your research is long, use the full_context=True parameter in CrewAI. Since Gemini has a 1M+ token window, you don't need to worry about the agents "forgetting" the start of the conversation.

## Parallel Workflows

In [110]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [111]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [112]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [113]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [114]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [115]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [116]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).

You are an agent. Your internal name is "TechResearcher".
-----------------------------------------------------------
Config:
{'http_options': {'headers': {'x-goog-api-client': 'google-adk/1.21.0 gl-python/3.12.11', 'user-agent': 'google-adk/1.21.0 gl-python/3.12.11'}}}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"Run the daily executive briefing on Tech, Health, and Finance"}],"role":"user"}
-----------------------------------------------------------
Functions:

----------------------------------------


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance


DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x30ac74e30>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x30ae7c9d0> server_hostname='generativelanguage.googleapis.com' timeout=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x30ae354c0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x30ae5d4d0> server_hostname='generativelanguage.googleapis.com' timeout=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x30ac74740>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x30ae040d0> server_hostname='generativelanguage.googleapis.com' timeout=None
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x30ae35550>
DEBUG:httpcore.http11:send_request_hea

DEBUG:urllib3.connectionpool:https://telemetry.crewai.com:4319 "POST /v1/traces HTTP/1.1" 200 2


In [117]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).

You are an agent. Your internal name is "TechResearcher".
-----------------------------------------------------------
Config:
{'http_options': {'headers': {'x-goog-api-client': 'google-adk/1.21.0 gl-python/3.12.11', 'user-agent': 'google-adk/1.21.0 gl-python/3.12.11'}}}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"Run the daily executive briefing on Tech, Health, and Finance"}],"role":"user"}
-----------------------------------------------------------
Functions:

----------------------------------------


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance


DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x30ae524b0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x30ae7c9d0> server_hostname='generativelanguage.googleapis.com' timeout=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x30ae4dd90>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x30ae5d4d0> server_hostname='generativelanguage.googleapis.com' timeout=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x30ae52e10>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x30ae040d0> server_hostname='generativelanguage.googleapis.com' timeout=None
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x30ae45dc0>
DEBUG:httpcore.http11:send_request_hea

DEBUG:urllib3.connectionpool:https://telemetry.crewai.com:4319 "POST /v1/traces HTTP/1.1" 200 2


---------

## Demo project: "Research Lab" Multi-Agent App

**Scenario**

For research and paper writing, a multi-agent system is far superior to a single AI because it mimics the **peer-review and editorial process**. Instead of one AI "guessing" the whole paper, you have specialized agents that fact-check, critique, and refine each other's work.


### The Agent Lineup 

 **The Librarian (Discovery Agent)**

* **Specialty:** Semantic search and source vetting.
* **Task:** Uses tools like ArXiv, Semantic Scholar, or Elicit APIs to find the top 10 most relevant papers. It doesn't just look for keywords; it looks for "methodological fit."
* **Output:** A curated list of PDFs and a "Relevance Score" for each.

**The Analyst (Extraction Agent)**

* **Specialty:** Data and Method synthesis.
* **Task:** Reads the papers found by the Librarian. It extracts specific data points, formulas, and limitations into a structured table.
* **Output:** A synthesis matrix (e.g., "Paper A used  method but had  sample size").

**The Ghostwriter (Drafting Agent)**

* **Specialty:** Academic tone and structure.
* **Task:** Takes the synthesis matrix and the user’s notes to draft specific sections (Introduction, Literature Review). It focuses on flow and logical transitions.
* **Output:** A rough Markdown draft.

**The Peer Reviewer (Critique Agent)**

* **Specialty:** Logical fallacies and "Red Teaming."
* **Task:** This agent is programmed to be **skeptical**. It looks at the Ghostwriter's draft and says: *"You claimed X, but the Analyst's data actually suggests Y. Also, your citation for Z is missing."*
* **Output:** A list of "Required Revisions."

---

### The Multi-Agent Workflow

In a single-agent app, you get one response. In this multi-agent app, the "magic" happens in the **loop**:

1. **Phase 1:** Librarian gathers  Analyst processes.
2. **Phase 2:** Ghostwriter drafts Section 1.
3. **Phase 3:** Peer Reviewer critiques Section 1.
4. **Phase 4:** If Peer Reviewer is unhappy, Ghostwriter **must** rewrite.
5. **Phase 5:** Final Polish agent ensures APA/MLA formatting and checks for "AI-style" repetitive phrasing.


**Benefits Over Single-Agent Systems**

| Feature | Single Agent (Standard Chat) | Multi-Agent "Research Lab" |
| --- | --- | --- |
| **Hallucinations** | High (often makes up citations). | Low (The Librarian provides the only "truth" the Writer can use). |
| **Tone** | Often "flowery" or generic. | Controlled by the "Style Enforcer" agent. |
| **Rigorousness** | Accepts its own first draft. | Forces internal "debate" before the user even sees it. |

---

**App Idea: "Manuscript Flow"**

A specialized workspace where your "Agents" live in a sidebar. You see a live "Activity Feed" of them talking to each other:

* *Librarian:* "Found a conflict in the data for the 2022 study."
* *Reviewer:* "Ghostwriter, please address this conflict in Paragraph 3."
* *Ghostwriter:* "Updating now..."

We will use CrewAI for this project



To build a multi-agent research app using only the **Google Generative AI SDK** (no CrewAI, LangGraph, etc.), you essentially need to manage a "State Machine" manually.

On the **Gemini Free Tier (2026)**, your best strategy is to use **Gemini 2.0 Flash** for most tasks because it offers a higher rate limit ( requests per day) compared to the Pro models, which are often restricted or removed from the free tier.

---

## Stage 1: The Orchestration Logic

Since you aren't using a framework, you must write a Python "Controller" that passes data between agents. Each "Agent" is actually just a specific **System Instruction** paired with a Gemini model instance.

### Step-by-Step Architecture:

1. **Define Agent "Personas":** Create a dictionary of system prompts (Librarian, Writer, Reviewer).
2. **State Management:** Use a simple Python dictionary to hold the "Paper State" (e.g., `current_draft`, `sources`, `critiques`).
3. **The Loop:** * Call **Agent A** (Librarian)  Save result to State.
* Pass State to **Agent B** (Writer)  Save draft to State.
* Pass State to **Agent C** (Reviewer)  Get feedback.
* If "Reviewer" flags errors, loop back to the Writer.



---

## Stage 2: Coding the Agents (Python ADK)

You will use the `google-generativeai` library. For the Free Tier, ensure you handle **Rate Limiting** by adding `time.sleep()` between calls if you hit 429 errors.

```python
import google.generativeai as genai
import os

# 1. Setup
genai.configure(api_key="YOUR_FREE_TIER_KEY")

# 2. Define the Agent Factories
def call_agent(role_instruction, user_input):
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=role_instruction
    )
    response = model.generate_content(user_input)
    return response.text

# 3. Specific Prompts
LIBRARIAN_PROMPT = "You are a research librarian. Extract key facts and citations from the provided text."
WRITER_PROMPT = "You are an academic writer. Use the facts provided to write a formal introduction."

```

---

## Stage 3: Implementing "Grounding"

A research app is useless if it hallucinates. Since you are on the Free Tier, you can use Gemini's **Built-in Google Search Tool** (Grounding). This allows your "Librarian" agent to actually browse the web for real papers.

```python
# Enable Google Search for the Librarian
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content(
    contents="What are the latest breakthroughs in fusion energy?",
    tools=[{'google_search': {}}]
)

```

---

## Stage 4: Managing Free Tier Constraints

In 2026, the Gemini Free Tier has specific "Token Buckets."

* **Gemini 2.0 Flash:** ~10-15 Requests Per Minute (RPM).
* **Context Window:** Use the **1M token window** to your advantage. Instead of many small calls, try to send the *entire* state of the research paper in one "Reviewer" call to save on your daily request quota (– RPD).

---

## Stage 5: User Interface (Streamlit)

Since you want an "app," the fastest way to wrap this Python logic without a heavy backend is **Streamlit**. It’s a single-file Python framework that creates a web UI for your agents.

| Feature | implementation |
| --- | --- |
| **Input** | A text box for the "Research Topic." |
| **Live Logs** | Display which agent is currently "thinking." |
| **Download** | Export the final `paper_state["final_draft"]` as a .docx or .txt file. |

**Would you like me to write a complete "Controller Loop" script that handles the handoff between the Librarian and the Writer?**

---

[Building a Research AI Agent from Scratch](https://www.youtube.com/watch?v=f3KHI1dpc1Q)
This video is highly relevant because it demonstrates how to build a multi-agent system from scratch using Python and Gemini without relying on complex orchestration frameworks.

In [16]:
!pip install "crewai[tools]" "langchain-google-genai" \
  "protobuf>=6.31.1,<7.0.0" \
  "google-auth<2.42.0,>=2.15.0" \
  "opentelemetry-api>=1.35.0" \
  "opentelemetry-sdk>=1.35.0,<1.39.0"

  Using cached google_auth-2.41.1-py2.py3-none-any.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of crewai[tools] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of google-genai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_google_genai-4.1.3-py3-none-any.whl.metadata (2.7 kB)
INFO: pip is still looking at multiple versions of google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.39.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.39.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached openteleme

In [ ]:
# run in google colab
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
import os

ModuleNotFoundError: No module named 'google.colab'

In [21]:
# run in vs code
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv


load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise RuntimeError("missing key")

In [34]:
# Initialize the Gemini Model (Flash is better for Free Tier speed/limits)
gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    verbose=True, 
    temperature=0.5
    
)

In [ ]:
topic

In [27]:
from crewai import Agent, Task, Crew, Process

# Agent 1: The Scholar
scholar = Agent(
  role='Senior Research Scholar',
  goal='Identify core arguments in {topic}',
  backstory='You are an expert at synthesizing academic papers.',
  llm=gemini_llm,
  allow_delegation=False
)

# Agent 2: The Writer
writer = Agent(
  role='Academic Ghostwriter',
  goal='Write a formal abstract for {topic}',
  backstory='You specialize in clear, peer-reviewed level prose.',
  llm=gemini_llm
)

# Task Definitions
task1 = Task(
    description='Summarize the last 3 years of {topic}.',
    agent=scholar,
    expected_output='{"type":"text","key":"summary"}',
)
task2 = Task(
    description='Write a 300-word abstract based on the summary.',
    agent=writer,
    expected_output='{"type":"text","key":"abstract"}',
)

# The Crew
research_crew = Crew(
  agents=[scholar, writer],
  tasks=[task1, task2],
  process=Process.sequential # Recommended for Free Tier to avoid hitting rate limits
)

result = research_crew.kickoff(inputs={'topic': 'Finance in 2026'})
print(result)

INFO:google_genai.models:AFC is enabled with max remote calls: 10.
DEBUG:httpcore.connection:connect_tcp.started host='generativelanguage.googleapis.com' port=443 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x16963c5c0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x16a67d4d0> server_hostname='generativelanguage.googleapis.com' timeout=None
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x16963cb30>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:urllib3.connectionpool:https://telemetry.cre

The period spanning 2023 to 2026 has profoundly reshaped global finance, driven by a complex interplay of macroeconomic re-calibration, accelerated technological integration, persistent geopolitical fragmentation, and a maturing sustainable investment agenda. By 2026, the financial system operates under a new paradigm defined by elevated capital costs and a fundamental re-evaluation of asset valuations. The 'higher-for-longer' interest rate environment, established through sustained central bank tightening, necessitated a re-pricing across equity, fixed income, and real estate, shifting focus towards corporate resilience, debt servicing capacity, and free cash flow generation in a less accommodative economic climate.

Technologically, Artificial Intelligence (AI) became an indispensable operational and strategic tool across all financial services, from advanced algorithmic trading and real-time risk management to hyper-personalized wealth advisory. This pervasive integration delivered 

DEBUG:urllib3.connectionpool:https://telemetry.crewai.com:4319 "POST /v1/traces HTTP/1.1" 200 2




╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                                                              │
│  Info: Tracing has been disabled.                                            │
│                                                                              │
│  Your preference has been saved. Future Crew/Flow executions will not        │
│  collect traces.                                                             │
│                                                                              │
│  To enable tracing later, do any one of these:                               │
│  • Set tracing=True in your Crew/Flow code                                   │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file               │
│  • Run: crewai traces enable                                                 │
│                                                                              │
╰─────────────────────────

In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
import os

# ---------------------------
# AGENT DEFINITIONS
# ---------------------------

# Agent 1: The Librarian (Discovery Agent)
librarian = Agent(
    role='Research Librarian & Source Discovery Expert',
    goal='Find and vet the most relevant academic papers for {topic}',
    backstory=(
        "You are an expert research librarian with a PhD in Information Science. "
        "You specialize in semantic search and source vetting using databases like "
        "ArXiv, Semantic Scholar, and Google Scholar. You don't just look for keywords; "
        "you evaluate papers based on methodological fit, citation impact, and "
        "relevance to the research question. You're skilled at distinguishing between "
        "high-quality peer-reviewed work and lower-quality sources."
    ),
    llm=gemini_llm,
    allow_delegation=False,
    verbose=True
)

# Agent 2: The Analyst (Extraction Agent)
analyst = Agent(
    role='Research Analyst & Data Synthesis Expert',
    goal='Extract and synthesize key data points, methods, and findings from research papers',
    backstory=(
        "You hold a PhD in Data Science and specialize in extracting structured information "
        "from academic papers. You're expert at identifying key methodologies, sample sizes, "
        "limitations, and statistical approaches. You create synthesis matrices that allow "
        "for easy comparison across studies, highlighting both consistencies and contradictions "
        "in the literature."
    ),
    llm=gemini_llm,
    allow_delegation=False,
    verbose=True
)

# Agent 3: The Ghostwriter (Drafting Agent)
ghostwriter = Agent(
    role='Academic Ghostwriter & Structural Expert',
    goal='Draft well-structured academic sections based on synthesized research',
    backstory=(
        "You are a professional academic writer with 15 years of experience helping "
        "researchers publish in top-tier journals. You specialize in creating clear, "
        "logically structured academic prose with proper flow and transitions. "
        "You're particularly skilled at turning complex research syntheses into "
        "accessible, well-organized sections that tell a compelling research story."
    ),
    llm=gemini_llm,
    allow_delegation=False,
    verbose=True
)

# Agent 4: The Peer Reviewer (Critique Agent)
peer_reviewer = Agent(
    role='Skeptical Peer Reviewer & Logical Fallacy Expert',
    goal='Critically evaluate drafts for logical consistency, evidence support, and academic rigor',
    backstory=(
        "You are a notoriously thorough journal editor known for your 'red team' approach. "
        "You actively look for weaknesses in arguments, missing citations, logical fallacies, "
        "and overstatements. You believe that rigorous critique improves research quality. "
        "You're programmed to be skeptical and demand high evidentiary standards."
    ),
    llm=gemini_llm,
    allow_delegation=False,
    verbose=True
)

# Agent 5: The Formatter (Polish Agent)
formatter = Agent(
    role='Academic Formatter & Style Editor',
    goal='Apply proper formatting and polish to the final document',
    backstory=(
        "You are a professional academic editor specializing in APA, MLA, Chicago, and "
        "other academic style guides. You have an eagle eye for formatting consistency, "
        "citation accuracy, and 'AI-tell' phrasing. You ensure the final document meets "
        "publication standards and reads as if written by a human expert."
    ),
    llm=gemini_llm,
    allow_delegation=False,
    verbose=True
)

# ---------------------------
# TASK DEFINITIONS
# ---------------------------

# Task 1: Literature Discovery
literature_discovery = Task(
    description=(
        "For the research topic: '{topic}', find the top 10-15 most relevant academic papers "
        "from the last 5 years. Focus on papers with strong methodological approaches and "
        "high citation impact. Evaluate each paper on:\n"
        "1. Relevance to the research question (0-10)\n"
        "2. Methodological rigor (0-10)\n"
        "3. Citation impact (0-10)\n"
        "4. Key findings\n"
        "5. Research gaps identified\n\n"
        "Create a curated bibliography with PDF links where available and a brief "
        "justification for each paper's inclusion."
    ),
    agent=librarian,
    expected_output=(
        "A structured report containing:\n"
        "1. Total papers reviewed\n"
        "2. Top 10-15 selected papers with full citations\n"
        "3. For each paper: Relevance Score, Method Score, Impact Score (all 0-10)\n"
        "4. Brief summary of why each paper was selected\n"
        "5. Overall research trends and gaps identified"
    ),
    output_file="literature_discovery.md"
)

# Task 2: Data Extraction & Synthesis
data_synthesis = Task(
    description=(
        "Using the papers identified by the Librarian, create a detailed synthesis matrix. "
        "For each paper, extract:\n"
        "1. Research question/hypothesis\n"
        "2. Methodology (quantitative/qualitative/mixed, specific methods used)\n"
        "3. Sample size and characteristics\n"
        "4. Key findings and conclusions\n"
        "5. Limitations acknowledged\n"
        "6. Statistical significance and effect sizes (if applicable)\n"
        "7. Theoretical framework\n\n"
        "Organize this into a comparative table that highlights:\n"
        "- Methodological consistencies/inconsistencies\n"
        "- Converging/diverging findings\n"
        "- Evolution of the field over time\n"
        "- Major gaps in the literature"
    ),
    agent=analyst,
    context=[literature_discovery],
    expected_output=(
        "A comprehensive synthesis matrix in table format with:\n"
        "1. Row for each paper with extracted data points\n"
        "2. Summary of methodological trends\n"
        "3. Summary of findings convergence/divergence\n"
        "4. Visual comparison of sample sizes, methods, etc.\n"
        "5. Identified research gaps and contradictions"
    ),
    output_file="data_synthesis.md"
)

# Task 3: Literature Review Draft
draft_literature_review = Task(
    description=(
        "Using the synthesis matrix from the Analyst, draft a comprehensive literature review "
        "section for a paper on '{topic}'. The literature review should:\n"
        "1. Begin with an introduction to the field\n"
        "2. Organize papers thematically/methodologically/chronologically\n"
        "3. Compare and contrast different studies\n"
        "4. Highlight methodological strengths and weaknesses\n"
        "5. Identify gaps in current research\n"
        "6. Lead logically to potential research questions\n"
        "7. Be approximately 1500-2000 words\n\n"
        "Ensure proper academic tone, logical flow between paragraphs, and "
        "appropriate use of transition words. Include in-text citations for all claims."
    ),
    agent=ghostwriter,
    context=[data_synthesis],
    expected_output=(
        "A well-structured literature review draft in markdown format with:\n"
        "1. Clear introduction establishing the field\n"
        "2. Thematic organization of existing research\n"
        "3. Critical analysis of methodologies and findings\n"
        "4. Clear identification of research gaps\n"
        "5. Logical conclusion that sets up future research\n"
        "6. Proper in-text citations throughout"
    ),
    output_file="literature_review_draft.md"
)

# Task 4: Peer Review Critique
peer_review_critique = Task(
    description=(
        "Critically review the literature review draft. Be thorough and skeptical. Identify:\n"
        "1. Any claims that lack sufficient evidence from the synthesis matrix\n"
        "2. Logical fallacies or overgeneralizations\n"
        "3. Missing citations for key claims\n"
        "4. Methodological critiques that should be included\n"
        "5. Alternative interpretations of the data\n"
        "6. Sections that need more elaboration\n"
        "7. Any potential biases in interpretation\n\n"
        "For each issue found, provide:\n"
        "- The specific problematic text\n"
        "- Why it's problematic\n"
        "- Specific suggestions for improvement\n"
        "- Supporting evidence from the synthesis matrix"
    ),
    agent=peer_reviewer,
    context=[draft_literature_review, data_synthesis],
    expected_output=(
        "A detailed peer review report containing:\n"
        "1. Overall assessment of the draft\n"
        "2. List of major issues (with specific examples)\n"
        "3. List of minor issues\n"
        "4. Specific revision suggestions\n"
        "5. Priority level for each revision (Critical/Important/Minor)\n"
        "6. Estimated time needed for revisions"
    ),
    output_file="peer_review_report.md"
)

# Task 5: Revised Draft (with iterative feedback loop)
revised_draft = Task(
    description=(
        "Rewrite the literature review based on the peer review feedback. Address ALL "
        "critical and important issues identified by the Peer Reviewer. Specifically:\n"
        "1. Add missing citations for all claims\n"
        "2. Strengthen weak arguments with evidence from the synthesis matrix\n"
        "3. Correct any logical fallacies or overgeneralizations\n"
        "4. Include methodological critiques where suggested\n"
        "5. Ensure balanced interpretation of the data\n"
        "6. Elaborate on sections marked as needing more detail\n\n"
        "Track all changes made in response to the peer review and provide a "
        "brief explanation of how each issue was addressed."
    ),
    agent=ghostwriter,
    context=[draft_literature_review, peer_review_critique],
    expected_output=(
        "A revised literature review draft with:\n"
        "1. All peer review issues addressed\n"
        "2. Change log documenting revisions\n"
        "3. Stronger evidence-based arguments\n"
        "4. More balanced and nuanced analysis\n"
        "5. Proper academic rigor throughout"
    ),
    output_file="revised_literature_review.md"
)

# Task 6: Final Formatting and Polish
final_polish = Task(
    description=(
        "Take the revised literature review and apply final formatting and polish:\n"
        "1. Apply {citation_style} formatting to all citations and references\n"
        "2. Check for and eliminate any 'AI-tell' repetitive phrasing\n"
        "3. Ensure consistent academic tone throughout\n"
        "4. Verify all citations match the bibliography\n"
        "5. Add proper section headings and formatting\n"
        "6. Create a properly formatted reference list\n"
        "7. Add a title page with appropriate metadata\n"
        "8. Ensure the document is publication-ready"
    ),
    agent=formatter,
    context=[revised_draft],
    expected_output=(
        "A fully polished, publication-ready literature review with:\n"
        "1. Proper {citation_style} formatting\n"
        "2. Consistent academic tone\n"
        "3. Complete and accurate reference list\n"
        "4. Professional formatting and layout\n"
        "5. No repetitive or unnatural phrasing"
    ),
    output_file="final_literature_review.md"
)

# ---------------------------
# CREW ASSEMBLY
# ---------------------------

# Create the Research Lab Crew
research_lab = Crew(
    agents=[librarian, analyst, ghostwriter, peer_reviewer, formatter],
    tasks=[
        literature_discovery,
        data_synthesis,
        draft_literature_review,
        peer_review_critique,
        revised_draft,
        final_polish
    ],
    process=Process.sequential,  # Sequential to mimic the research workflow
    verbose=True,
    memory=True  # Enable memory so agents can reference previous work
)

# ---------------------------
# EXECUTION
# ---------------------------

if __name__ == "__main__":
    # Input parameters
    research_topic = "The impact of AI on academic research productivity"
    citation_style = "APA 7th edition"  # Can be changed to MLA, Chicago, etc.
    
    # Execute the crew
    print(f"Starting Research Lab for topic: {research_topic}")
    print("=" * 60)
    
    inputs = {
        'topic': research_topic,
        'citation_style': citation_style
    }
    
    result = research_lab.kickoff(inputs=inputs)
    
    print("\n" + "=" * 60)
    print("RESEARCH COMPLETE!")
    print("=" * 60)
    print("\nOutput files created:")
    print("1. literature_discovery.md - Initial paper discovery")
    print("2. data_synthesis.md - Synthesis matrix")
    print("3. literature_review_draft.md - First draft")
    print("4. peer_review_report.md - Critique feedback")
    print("5. revised_literature_review.md - Revised draft")
    print("6. final_literature_review.md - Polished final version")
    print("\nFinal result summary:")
    print(result)

ERROR:crewai.utilities.llm_utils:Error instantiating LLM from unknown object type: Error importing native provider: OPENAI_API_KEY is required


ImportError: Error importing native provider: OPENAI_API_KEY is required

In [35]:
from crewai import Agent, Task, Crew, Process

# Initialize Gemini LLM (make sure you have the proper setup)
# Assuming gemini_llm is already defined as per your code

### AGENT DEFINITIONS ###

# Agent 1: The Librarian (Discovery Agent)
librarian = Agent(
    role='Academic Librarian',
    goal='Find top 10 most relevant papers for {topic} using semantic search',
    backstory='Expert in semantic search and source vetting. Uses methodological fit rather than just keywords.',
    llm=gemini_llm,
    allow_delegation=False
)



# Agent 2: The Analyst (Extraction Agent)
analyst = Agent(
    role='Research Analyst',
    goal='Extract data, formulas, and limitations from papers into structured synthesis matrix',
    backstory='Specializes in data and method synthesis from academic literature.',
    llm=gemini_llm,
    allow_delegation=False
)

# Agent 3: The Ghostwriter (Drafting Agent)
ghostwriter = Agent(
    role='Academic Ghostwriter',
    goal='Draft specific sections (Introduction, Literature Review) from synthesis matrix',
    backstory='Specializes in academic tone, structure, and logical flow.',
    llm=gemini_llm,
    allow_delegation=False
)

# Agent 4: The Peer Reviewer (Critique Agent)
reviewer = Agent(
    role='Skeptical Peer Reviewer',
    goal='Critique drafts for logical fallacies, missing citations, and inconsistencies',
    backstory='Programmed to be skeptical. Identifies gaps between claims and evidence.',
    llm=gemini_llm,
    allow_delegation=False
)

# Agent 5: The Polisher (Formatting Agent)
polisher = Agent(
    role='Academic Polisher',
    goal='Ensure proper formatting (APA/MLA) and remove AI-style repetitive phrasing',
    backstory='Expert in academic formatting standards and natural academic prose.',
    llm=gemini_llm,
    allow_delegation=False
)

### TASK DEFINITIONS ###

# Phase 1: Discovery
task_discovery = Task(
    description='Find the top 10 most relevant papers for {topic}. Focus on methodological fit. Provide PDF links and Relevance Score for each.',
    agent=librarian,
    expected_output='Curated list of 10 papers with URLs and Relevance Scores (1-10)'
)

# Phase 1: Analysis
task_analysis = Task(
    description='Read the papers from the Librarian. Extract data points, formulas, limitations into structured synthesis matrix: Paper, Method, Sample Size, Key Findings, Limitations.',
    agent=analyst,
    expected_output='Synthesis matrix table with columns: Paper, Method, Sample Size, Key Findings, Limitations'
)

# Phase 2: Drafting (with loop capability)
task_draft_intro = Task(
    description='Using the synthesis matrix from the Analyst, draft the Introduction section for {topic}.',
    agent=ghostwriter,
    expected_output='Rough Markdown draft of Introduction section'
)

# Phase 3: Review (Critique)
task_review_intro = Task(
    description='Critique the Introduction draft. Identify: 1) Claims not supported by Analyst data, 2) Missing citations, 3) Logical gaps.',
    agent=reviewer,
    expected_output='List of Required Revisions with specific issues found'
)

# Phase 4: Rewrite (Conditional - only if revisions needed)
task_rewrite_intro = Task(
    description='Rewrite the Introduction section addressing ALL revisions from the Peer Reviewer.',
    agent=ghostwriter,
    expected_output='Revised Introduction section'
)

# Phase 5: Final Polish
task_polish_final = Task(
    description='Apply APA/MLA formatting to the final paper. Remove repetitive phrasing and ensure academic tone.',
    agent=polisher,
    expected_output='Final polished paper with proper formatting'
)

### THE WORKFLOW CREW ###

# Main research crew with sequential process
research_crew = Crew(
    agents=[librarian, analyst, ghostwriter, reviewer, polisher],
    tasks=[
        task_discovery,
        task_analysis,
        task_draft_intro,
        task_review_intro,
        task_rewrite_intro,
        task_polish_final
    ],
    process=Process.sequential,
    verbose=True
)

# Execute the workflow
result = research_crew.kickoff(inputs={'topic': 'Finance in 2026'})
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6a737059-9b64-4105-89f5-169a7d02569b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

INFO:google_genai.models:AFC is enabled with max remote calls: 10.
DEBUG:httpcore.connection:connect_tcp.started host='generativelanguage.googleapis.com' port=443 local_address=None timeout=None socket_options=None


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Academic Librarian                                                                                      │
│                                                                                                                 │
│  Task: Find the top 10 most relevant papers for Finance in 2026. Focus on methodological fit. Provide PDF       │
│  links and Relevance Score for each.                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/e5028514/code/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x16963fe30>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x17ffa67d0> server_hostname='generativelanguage.googleapis.com' timeout=None
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x309097320>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 400, b'Bad Request', [(b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Type', b'application/json; charset=UTF-8'), (b'Content-Encoding', b'gzip'), (b'D

...

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



╭────────────────────────────── Execution Traces ──────────────────────────────╮
│                                                                              │
│  🔍 Detailed execution traces are available!                                 │
│                                                                              │
│  View insights including:                                                    │
│    • Agent decision-making process                                           │
│    • Task execution flow and timing                                          │
│    • Tool usage details                                                      │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
Would you like to view your execution traces? [y/N] (20s timeout): 

DEBUG:urllib3.connectionpool:https://telemetry.crewai.com:4319 "POST /v1/traces HTTP/1.1" 200 2




╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                                                              │
│  Info: Tracing has been disabled.                                            │
│                                                                              │
│  Your preference has been saved. Future Crew/Flow executions will not        │
│  collect traces.                                                             │
│                                                                              │
│  To enable tracing later, do any one of these:                               │
│  • Set tracing=True in your Crew/Flow code                                   │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file               │
│  • Run: crewai traces enable                                                 │
│                                                                              │
╰─────────────────────────